<a href="https://colab.research.google.com/github/FeelOneE/Kaggle_study/blob/main/Spooky_Author_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/kaggle
% ls -al

/content/drive/MyDrive/kaggle
total 6396
-rw------- 1 root root      65 Jan 13 09:17 kaggle.json
-rw------- 1 root root   30082 Jan 13 09:23 sample_submission.zip
-rw------- 1 root root 1351241 Oct 25  2017 test.csv
-rw------- 1 root root  551200 Jan 13 09:23 test.zip
-rw------- 1 root root 3295644 Oct 25  2017 train.csv
-rw------- 1 root root 1319237 Jan 13 09:23 train.zip


# 캐글 데이터 세트 로드

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         76           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        107           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        909           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        177           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1896           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      13380            True  
house-pr

In [ ]:
!kaggle competitions download -c spooky-author-identification

  0% 0.00/29.4k [00:00<?, ?B/s]
100% 29.4k/29.4k [00:00<00:00, 3.74MB/s]
  0% 0.00/538k [00:00<?, ?B/s]
100% 538k/538k [00:00<00:00, 29.8MB/s]
  0% 0.00/1.26M [00:00<?, ?B/s]
100% 1.26M/1.26M [00:00<00:00, 83.5MB/s]


In [ ]:
! unzip test.zip

Archive:  test.zip
  inflating: test.csv                


In [ ]:
! unzip train.zip

Archive:  train.zip
  inflating: train.csv               


In [3]:
! ls -al

total 6396
-rw------- 1 root root      65 Jan 13 09:17 kaggle.json
-rw------- 1 root root   30082 Jan 13 09:23 sample_submission.zip
-rw------- 1 root root 1351241 Oct 25  2017 test.csv
-rw------- 1 root root  551200 Jan 13 09:23 test.zip
-rw------- 1 root root 3295644 Oct 25  2017 train.csv
-rw------- 1 root root 1319237 Jan 13 09:23 train.zip


In [5]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [6]:
df_test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


#토크나이저 로딩

1. 최대 길이 초과 제거
2. 고정 길이 패딩
3. 어텐션 마스크 만들기


In [3]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 79.7 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
     |████████████████████████████████| 895 kB 82.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [9]:
# 자연어 데이터 토큰 아이디로 인코드 하기
encoded_arr = [ tokenizer.encode(x ) for x in df_train['text'] ]
len(encoded_arr)

Token indices sequence length is longer than the specified maximum sequence length for this model (955 > 512). Running this sequence through the model will result in indexing errors


19579

In [10]:
len(encoded_arr[0])

54

In [11]:
# 최대 길이 보기
max( list( len(x) for x in encoded_arr ) )

955

In [12]:
# 길이 512 넘어가는 문장 삭제
new_encoded_arr = []
for i in encoded_arr:
  if len(i) <= 512 :
    new_encoded_arr.append(i)
len(new_encoded_arr)

19577

In [13]:
# 최대 길이 보기
max( list( len(x) for x in new_encoded_arr ) )

504

In [14]:
# 데이터 패딩

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    new_encoded_arr, padding="post", maxlen=512
)

In [15]:
len(padded_inputs[0])

512

In [16]:
# 학습 속도를 높이기 위한 어텐션 마스크 표시 
def attention_mask(ids): 
  masks = [] 
  for id in ids: 
    mask = [float(i>0) for i in id] 
    masks.append(mask) 
  
  return masks



# BERT 모델 로드

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

------------------------------- 참고 내용 -----------------------------------------
https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

In [5]:
# Read data
data = pd.read_csv("train.csv")
data.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [6]:
# Define pretrained tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
#model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
# ----- 1. Preprocess data -----#
# Preprocess data
X = list(data["text"])
y = list(data["author"])

# x_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512, add_special_tokens = True,return_attention_mask=True)
# x_test_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
np_y = le.transform(y)
np_y[:30]

array([0, 1, 0, 2, 1, 2, 0, 0, 0, 2, 2, 0, 1, 1, 0, 2, 0, 2, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, np_y, test_size=0.2)

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenizer(x_train, padding=True, truncation=True, max_length=512, add_special_tokens = True,return_attention_mask=True, return_tensors="tf")),  # Convert BatchEncoding instance to dictionary
    y_train
)).shuffle(1000).batch(32).prefetch(1)

validation_dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenizer(x_test, padding=True, truncation=True, max_length=512, return_tensors="tf")),
    y_test
)).batch(32).prefetch(1)

In [16]:
train_dataset


<PrefetchDataset shapes: ({input_ids: (None, 512), token_type_ids: (None, 512), attention_mask: (None, 512)}, (None,)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>

In [28]:
cnt = 0
for i in train_dataset:
  print(i)
  print('-'*20)
  print(i[0]['input_ids'][0])
  #print(i[0]['token_type_ids'])
  print(i[0]['attention_mask'][0])
  #print('-'*20)
  #print(i[1])
  cnt += 1
  if cnt > 0:
    break

({'input_ids': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[ 101, 1999, 1037, ...,    0,    0,    0],
       [ 101, 2012, 1996, ...,    0,    0,    0],
       [ 101, 1000, 2002, ...,    0,    0,    0],
       ...,
       [ 101, 2182, 1996, ...,    0,    0,    0],
       [ 101, 2035, 2008, ...,    0,    0,    0],
       [ 101, 2085, 2130, ...,    0,    0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}, <tf.Tensor: shape=(32,), dtype=in

---- 모델링 ----

In [12]:
from transformers import TFBertForSequenceClassification, AdamW, BertConfig


# 참고 자료
#
## https://stackoverflow.com/questions/60463829/training-tfbertforsequenceclassification-with-custom-x-and-y-data
#
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

#model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5

learning_rate = 2e-5

# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x=train_dataset,
          y=None, 
          validation_data=validation_dataset,
          epochs=number_of_epochs,
          batch_size=32 ,
          )

 22/490 [>.............................] - ETA: 3:03:41 - loss: 1.0495 - accuracy: 0.4432

KeyboardInterrupt: ignored

In [ ]:
len(X_train_tokenized['input_ids'][0])

512

In [ ]:
from sklearn.model_selection import train_test_split

# label, train data 개수 맞춰야함

labels = data.author.values
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(X_train_tokenized['input_ids'], labels,random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(X_train_tokenized['attention_masks'], X_train_tokenized['input_ids'],random_state=56, test_size=0.2)

ValueError: ignored

In [ ]:
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)

AttributeError: ignored